<a href="https://colab.research.google.com/github/Qasim-Chaudhry/Qasim-Chaudhry/blob/main/Parameter_Efficient_Supervised_Fine_Tuning_of_LLaMA_3_2_(3B)_on_a_Medical_Chain_of_Thought_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1**

import the libraries

In [2]:
!pip install unsloth wandb datasets rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━

In [1]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install trl
!pip install sentencepiece
!pip install scipy

**Step 2**

in this i import weight and biases

In [2]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: qasimafraz88 (qasimafraz88-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

**Step 3**

Load the data set import from hugging face

In [3]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en')
train_data = dataset['train'].select(range(100, len(dataset['train'])))
val_data = dataset['train'].select(range(0, 100))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]


**Step 4**

*   FastLanguageModel is a tool from the Unsloth library that allows you to load
LLaMA models efficiently for training or inference.
*  model_name = "unsloth/Llama-3.2-3B" tells the code to load the pre-trained LLaMA 3.2 (3B) model hosted on Hugging Face.




In [6]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B",
    load_in_4bit = True,
    max_seq_length = 2048,
    dtype = None,
    device_map = "auto",
)


NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

**Step 5**

*   parameter-efficient fine-tuning (PEFT) to the loaded model using LoRA, allowing
it to fine-tune only a small subset of weights to save memory and speed up training.



In [ ]:
model = FastLanguageModel.get_peft_model(model)


Unsloth 2025.5.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


**Step 6**

*  medical question-answer pairs with tags, tokenizes them, and creates labels for model training.



In [ ]:
def format_input(example):
    text = f"<think>{example['Question']}</think> <response>{example['Complex_CoT']}</response>"
    # Remove padding='longest' from here
    tokenized = tokenizer(text, truncation=True)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # Create labels, ignore padding tokens (-100)
    # Labels should have the same length as input_ids
    labels = [tid if tid != tokenizer.pad_token_id else -100 for tid in input_ids]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

In [ ]:
train_data = train_data.map(format_input)
val_data = val_data.map(format_input)

Map:   0%|          | 0/19604 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

**Step 8**

*   collate_batch function pads input IDs, attention masks, and labels in a batch to the same length for efficient model training.



In [ ]:
def collate_batch(batch):
    input_ids = [example["input_ids"] for example in batch]
    attention_mask = [example["attention_mask"] for example in batch]
    labels = [example["labels"] for example in batch]

    return {
        "input_ids": torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id),
        "attention_mask": torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0),
        "labels": torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100),
    }


In [ ]:
# #def collate_batch(batch):
#  #   texts = [example["text"] for example in batch]
#   #  return tokenizer(
#    #     texts,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=512,
#     )
def collate_batch(batch):
    texts = [example["Question"] for example in batch]
    encodings = tokenizer(
        texts,
        padding=True,          # <--- Ensures equal length
        truncation=True,       # <--- Cuts off very long inputs
        return_tensors="pt"    # <--- Returns PyTorch tensors
    )
    # Dummy labels if you're doing language modeling or a placeholder for now
    labels = encodings["input_ids"].clone()  # Adjust as needed for your task
    encodings["labels"] = labels
    return encodings


**Step 9**

*   CoTDataset class prepares tokenized input data (input IDs, attention masks, and labels) for training by converting it into a PyTorch-compatible dataset format



In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class CoTDataset(Dataset):
    def __init__(self, data):
        self.input_ids = data["input_ids"]
        self.attention_mask = data["attention_mask"]
        self.labels = data["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_mask[idx]),
            "labels": torch.tensor(self.labels[idx])
        }
# class CoTDataset(Dataset):
#     def __init__(self, data):
#         self.data = data  # This is a pandas DataFrame

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return {"Question": self.data.iloc[idx]["Question"]}


In [ ]:
# train_loader = DataLoader(CoTDataset(train_data), batch_size=1, shuffle=True, collate_fn=collate_batch)
# val_loader = DataLoader(CoTDataset(val_data), batch_size=1, collate_fn=collate_batch)
train_loader = DataLoader(CoTDataset(train_data), batch_size=2, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(CoTDataset(val_data), batch_size=2, collate_fn=collate_batch)


**Step 10**

*   PyTorch DataLoaders for training and validation using a sequence-to-sequence data collator that dynamically pads the input data.



In [ ]:
from transformers import DataCollatorForSeq2Seq

collate_fn = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")
train_loader = DataLoader(train_data, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=2, collate_fn=collate_fn)


**Step 11**

*   Weights & Biases tracking for the project and sets up the AdamW optimizer with a learning rate of 2e-4 for model training."



In [ ]:
wandb.init(project="llama3-medical-finetune")
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-4)

In [ ]:
device = torch.device("cuda" )
model.to(device)
model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

**Step 12**

*  Code trains the model for 3 epochs, computes loss on each batch, updates weights, and logs average training loss per epoch to Weights & Biases.


In [1]:
for epoch in range(3):
    total_loss = 0
    print(f" Starting epoch {epoch+1}")
    for i, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        if i % 10 == 0:
            print(f"Batch {i+1}: Loss = {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    wandb.log({"epoch": epoch, "train_loss": avg_loss})
    print(f"Epoch {epoch+1} complete | Avg Loss: {avg_loss:.4f}")

 Starting epoch 1


NameError: name 'train_loader' is not defined

In [ ]:
save_path = "llama3_medical_finetuned"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f" Model & tokenizer saved at: {save_path}")